In [2]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model, Sequential
import tensorboard
import os

In [3]:
# Model 1: from research paper. training data is 72096 images whereas the validation is 3799
# this means that the validation data is only about 5% of the training data which is insufficient.
# Therefore, in this model, we are providing a 25% split on the training images to be used as validation data

In [4]:
batch_size = 32
img_height = 128
img_width = 128

In [4]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding= 'same', input_shape=(img_height, img_width, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding= 'same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding= 'same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=8, activation = 'softmax'))

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 64)       0

In [5]:
# load the augmented data from the local data directory:

train_dir = "Data/Processed_Data/train"
test_dir = "Data/Processed_Data/test"

datagen = ImageDataGenerator(rescale = 1./255, validation_split= 0.25) # The imported data will br normalized here.
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123, subset = 'training') # set as training data

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123, subset = 'validation') # set as validation data

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123,) # set as testing data

Found 54072 images belonging to 8 classes.
Found 18024 images belonging to 8 classes.
Found 3799 images belonging to 8 classes.


In [7]:
r = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_steps=validation_generator.samples//batch_size,
    validation_data = validation_generator,
    epochs = 50,
)

C:\Users\GHAYAAS\AppData\Local\Temp/ipykernel_16344/3220806991.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
1689/1689 [==============================] - 183s 99ms/step - loss: 1.6460 - accuracy: 0.3833 - val_loss: 1.6240 - val_accuracy: 0.3996
Epoch 2/50
1689/1689 [==============================] - 138s 82ms/step - loss: 1.2910 - accuracy: 0.5294 - val_loss: 1.3908 - val_accuracy: 0.4820
Epoch 3/50
1689/1689 [==============================] - 129s 76ms/step - loss: 1.0470 - accuracy: 0.6103 - val_loss: 1.3595 - val_accuracy: 0.4999
Epoch 4/50
1689/1689 [==============================] - 129s 76ms/step - loss: 0.9263 - accuracy: 0.6514 - val_loss: 1.3481 - val_accuracy: 0.5185
Epoch 5/50
1689/1689 [==============================] - 134s 79ms/step - loss: 0.8409 - accuracy: 0.6804 - val_loss: 1.2153 - val_accuracy: 0.5475
Epoch 6/50
1689/1689 [==============================] - 118s 70ms/step - loss: 0.7835 - accuracy: 0.6997 - val_loss: 1.4436 - val_accuracy: 0.5235
Epoch 7/50
1689/1689 [==============================] - 117s 69ms/step - loss: 0.7385 - accuracy: 0.7160 - val_loss: 1

In [1]:
#model.save('model_researchpaper_2019_2.h5')

In [6]:
#model = tf.keras.models.load_model('model_researchpaper_2019_2.h5')

In [7]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['ak', 'bcc', 'bkl', 'df', 'mel','nv', 'scc', 'vasc']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

C:\Users\GHAYAAS\AppData\Local\Temp/ipykernel_12364/2098712288.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)


In [8]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_generator.classes, y_pred))

0.07501974203737825


In [ ]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(test_generator.classes, y_pred))